In [3]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.tsa.arima.model import ARIMA
import pmdarima as pm
from sklearn.metrics import r2_score
import warnings 

In [4]:
with open("sp500_list.txt", "r") as f:
    sp500_list = [line.strip() for line in f]

In [5]:
rfdata = pd.read_csv("market_data.csv", index_col=0, low_memory=False)
rfdata = rfdata.drop(rfdata.index[[0,1]])
rfdata.columns = pd.MultiIndex.from_arrays([["Adj Close"]*503 + ["Volume"]*503, sp500_list*2])
rfdata.index = pd.core.indexes.datetimes.DatetimeIndex(rfdata.index)
rfdata = rfdata.astype(float)

data = rfdata["Adj Close"]
data = data.dropna(axis=1)

In [6]:
data=data.dropna(axis=1)

In [19]:
%%time

score_train_arima = []
score_test_arima = []
order = []
arima7_pred = pd.DataFrame(columns=data.columns)

for i in data.columns:
    frame = data[i].resample('W').last().pct_change().dropna()
    
    train= frame["2013-01-01":"2023-01-01"]
    test = frame["2023-01-08":]
    
    model = pm.auto_arima(train, max_p=3, max_q=3)
    mod = ARIMA(train, order=model.order)
    res = mod.fit()
    
    pred=[]
    for j in range(len(test.index)):
        if j == 0:
            pred.append(res.forecast().iloc[0])
        else:
            pred.append(res.extend(list(test[0:j])).forecast().iloc[0])
    
    score_test_arima.append(r2_score(test, pred))
    score_train_arima.append(r2_score(train, res.predict(start="2013-01-06", end ="2023-01-01" )))
    order.append(model.order)
                                                  
    arima7_pred[i] = pred
    

arima7_score = pd.DataFrame({"score_train" : score_train_arima, "score_test": score_test_arima, "model" : order}, data.columns)

CPU times: total: 1h 2min 42s
Wall time: 1h 8min 43s


In [27]:
#arima7_score.to_csv("ARIMA7score.csv")
#arima7_pred.to_csv("ARIMA7pred.csv")